In [ ]:
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [2]:
import re
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [68]:
torch.manual_seed(2023)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(2023)

In [ ]:
!pip install datasets

from datasets import load_dataset, Dataset, DatasetDict

In [6]:
dataset = load_dataset("IgorVolochay/russian_jokes")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

'После проведения акции "К животным по-человечески" животные посовещались и решили провести акцию "К человеку по-скотски".'

In [7]:
dataset["train"]["text"][1]

'После проведения акции "К животным по-человечески" животные посовещались и решили провести акцию "К человеку по-скотски".'

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [35]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(10000))

In [36]:
train_path = 'train_dataset.txt'
with open(train_path, "w") as f:
    f.write('\n'.join(small_train_dataset["text"]) + '\n')

In [27]:
model_name = "sberbank-ai/rugpt3medium_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name).to('cuda:0')

In [57]:
def generate(input_text):
  input_ids = tokenizer.encode(input_text, return_tensors="pt").to('cuda:0')

  model.eval()
  with torch.no_grad():
      out = model.generate(
          input_ids,
          do_sample=True,
          num_beams=3,
          temperature=1.5,
          top_p=2.2,
          max_length=50,
          repetition_penalty=1.2,
    )
  generated_text = list(map(tokenizer.decode, out))[0]
  print(generated_text)

In [38]:
train_dataset = TextDataset(tokenizer=tokenizer, file_path=train_path, block_size=32)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [39]:
generate("Встречаются два друга:")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Встречаются два друга: - Слушай, у меня появилась идея! Мы с тобой можем пожениться! Через некоторое время они снова встретились. - Слушай, все-таки решил жениться? - Да нет, я передумал. - А чего так? - Ну, я же не знаю, как ты к этому относишься. - Ну, тогда женись на мне. - Не хочу! - Ну, тогда на мне. - Не хочу! - Ну, тогда на мне! - Не хочу! - Ну


Модель до дообучения выдает не слишком похожие на анекдот результаты.

In [42]:
training_args = TrainingArguments(
    output_dir="./finetuned",         # The output directory
    overwrite_output_dir=True,        # Overwrite the content of the output dir
    num_train_epochs=5,              # number of training epochs
    per_device_train_batch_size=20,   # batch size for training
    per_device_eval_batch_size=32,    # batch size for evaluation
    warmup_steps=9,                   # number of warmup steps for learning rate scheduler
    gradient_accumulation_steps=5
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(), lr=1e-5), None)
)

In [43]:
trainer.train()

Step,Training Loss
500,3.594100


TrainOutput(global_step=640, training_loss=3.5707955837249754, metrics={'train_runtime': 1441.1771, 'train_samples_per_second': 44.641, 'train_steps_per_second': 0.444, 'total_flos': 3711784500854784.0, 'train_loss': 3.5707955837249754, 'epoch': 4.97})

In [44]:
model.save_pretrained('model_jokes')

In [55]:
generate("Встречаются два друга:")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Встречаются два друга: - Я смотрю, ты уже нашел свою любовь... - Да, нашел. - А как она к тебе относится? - Никак


In [58]:
generate("Штирлиц пришел домой и")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Штирлиц пришел домой и увидел, что жена приготовила ему ужин. - А почему у тебя только одно яйцо? - удивился он. - Яйцо я кладу в морозилку, а вот второе я достаю, когда ем.


In [47]:
generate("Пессимисты")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Пессимисты, конечно, пессимисты, но что делать, если мы живем в стране, где все знают, что такое смерть.
- Ты что, в самом деле хочешь, чтобы я переспал с тобой?- Нет, просто мне кажется, что лучше, если ты переспишь со мной, чем с кем-то другим.
- В чем разница между русским и украинцем?- Русский никогда не говорит, что он русский. Он говорит, что он русский.
Разговаривают два


In [66]:
generate("Это прачечная?")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Это прачечная? - Да. - А где, по-вашему, находится туалет? - По-моему, в туалете.
- Как называется эта птица, которую вы никогда не видели?- Это я.
Мужик приходит к любовнице


**Вывод**

Видно, что дообученная модель лучше понимает структуру и различные виды анекдотов более детально. Но юмор в этих анекдотах чересчур специфичный и не всегда понятный.